In [1]:
from Sample import Sample
from registration.register import FeatureExtractor
from preprocessing.preprocess import preprocess
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd


In [2]:
in_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
out_path="/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/REG_IMConIF"
samples=['15-3531', '04-0164', '14-0025', '14-2675', '18-1462', '16-1370']

In [3]:
for s in samples:  

    print('Sample: ' + s)

    S = Sample(glob.glob(in_path + '/*{}*'.format(s))[0])

    out = os.path.join(out_path, s)
    if not os.path.exists(out):
        os.makedirs(out)

    df = pd.DataFrame(columns = ['RoI', 'delta_x', 'delta_y'])

    for i, roi in enumerate(S.rois):

        print('RoI_' + str(roi.roi_num) + ': ')
        ppIF = preprocess(roi.if_nuc)
        ppIMC = preprocess(roi.imc_nuc)

        ex = FeatureExtractor("sift")
        ex(ppIMC, ppIF)
        ex.match()
        ex.draw_matches()
        ex.estimate()
        ex.warp()
        im = ex.compute_overlay()

        x, y = ex.calc_misalignment()

        print('delta_x= ' + str(x) + ' delta_y= ' + str(y))
        df.loc[i] = [roi.roi_num, x, y]

        plt.imsave(os.path.join(out, s + '_RoI_' + str(roi.roi_num) + '.png'), im)

    df.to_csv(out + '/relocation_result.csv', index=False)

Sample: 15-0813


Exception: No ROIs found in '/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/20220203_15-0813_BM'

In [6]:
import os
import pandas as pd


df = pd.DataFrame(columns = ['Sample', 'mean_x', 'mean_y', 'mean_all'])

for i,f in enumerate(samples):
    tmp = pd.read_csv(os.path.join(out_path, f, 'relocation_result.csv'))
    x_mean = tmp['delta_x'].mean()
    y_mean = tmp['delta_y'].mean()
    overall_mean = (x_mean + y_mean)/2
    df.loc[i] = [f, x_mean, y_mean, overall_mean]

df.to_csv(out_path + '/relocation_evaluation_summary_5.csv', decimal=',', sep=';', index=False)